<a href="https://colab.research.google.com/github/timsetsfire/wandb-examples/blob/main/colab/Custom_Plots_from_History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb

In [2]:
import wandb
import numpy as np
from typing import TYPE_CHECKING, Any

class CustomChartFromHistory(wandb.viz.CustomChart):
    @staticmethod
    def user_query(table_name) -> dict[str, Any]:
        return {
            "queryFields": [
                {
                    "name": "runSets",
                    "args": [
                        {"name": "runSets", "value": "${runSets}"},
                        {"name": "limit", "value": 500}
                        ],
                    "fields": [
                        {"name": "id", "fields": []},
                        {"name": "name", "fields": []},
                        {"name": "_defaultColorIndex", "fields": []},
                        {
                            "name": "history",
                            "args": [{"name": "keys", "value": CustomChartFromHistory.fields },],
                            "fields": [],
                        },
                    ],
                }
            ],
        }

def log_custom_history_plot(wandb_run, vega_spec, panel_name, fields, str_fields):
  CustomChartFromHistory.fields = list(fields.values())
  # fields = dict(upper = "upper", lower = "lower", center = "center", index = "step")
  # str_fields = dict(title = "x with errors bars")
  chart = CustomChartFromHistory(vega_spec, None, fields, str_fields)
  wandb_run.log({panel_name: chart})

In [3]:
import numpy as np

wandb.init(project = "error-bar-plotting-v3", entity = "tim-w")

## you would run this for each error bar you want to plot in wandb.
log_custom_history_plot(wandb.run, "tim-w/error-bands-3", "error-plots/x_plot", dict(center = "x", upper = "x_upper", lower = "x_lower", index = "step"), str_fields = dict(title = "x +/- std"))
log_custom_history_plot(wandb.run, "tim-w/error-bands-3", "error-plots/y_plot", dict(center = "y", upper = "y_upper", lower = "y_lower", index = "step"), str_fields = dict(title = "y +/- std"))

for i in range(100):
  x = np.random.randn()
  y = np.random.randn()
  x_std = np.random.rand() / 2
  y_std = np.random.rand() * 2
  wandb.log(dict(x = x, x_upper = x+x_std, x_lower = x-x_std, step = i, y = y, y_upper = y+y_std, y_lower = y-y_std))
wandb.finish()

wandb: Currently logged in as: tim-w. Use `wandb login --relogin` to force relogin


step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
x,▄▅▅█▁▅█▃▂▃▃▆▄▁▇▆▅▄▅▆▄▃▃▆▅▃▆▂▆▃▅▅▆▅▄▇▆█▅▆
x_lower,▃▅▅█▁▅▇▃▂▃▃▅▃▁▇▆▄▄▅▆▄▂▃▆▅▃▆▁▆▃▄▅▆▅▄▆▅█▄▅
x_upper,▄▄▅█▁▅█▂▂▄▃▆▄▁█▆▅▅▆▆▅▃▃▆▆▂▆▃▅▄▅▅▆▆▅▇▆▇▅▆
y,▅█▆▆▄▃▃▃▆▅▃▆█▄▄▆▅▁▆▄▆▄▇▆▄▅▄▅▄▇▇▇▅▅▆▆▄▃▆▄
y_lower,▃▇▆▆▅▁▃▁▄▄▂▅█▄▃▆▅▂▄▅▆▂▅▅▄▃▃▃▃▇▇▆▄▆▆▅▃▁▆▃
y_upper,▆▇▆▅▄▅▃▅▇▆▅▇▇▅▅▅▅▁▇▄▅▆█▇▄▆▅▆▅▆▆█▅▅▅▆▅▄▅▅
step,99
x,0.73614
x_lower,0.40863
x_upper,1.06366
